In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as image
import os
import cv2

In [ ]:
capture = cv2.VideoCapture(0)

In [ ]:
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_alt.xml')

In [ ]:
#Iterate to save data for each face
face_data = []
face_name = input("Enter face name :")

face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_alt.xml')
dataset_path = './data/'

#non-unique count, faces captured in total
facecount = 0
while True:
    #ret is a return code, frame is the frame captured
    ret, frame = capture.read()
    #convert to grayscale if needed
    grayscale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #ret is false if frame was not captured properly, so we ignore that particular frame
    if ret == False:
        continue
    #stores the faces coordinates in rectangle start, width and height
    faces = face_cascade.detectMultiScale(grayscale, 1.3, 5)
    #iterating over each face detected in that particular frame
    for face in faces:
        #extracting our values from face
        x,y,w,h = face
        #offset value to increase our displayed rectangle size
        offset = 50
        #creating a rectangle using the coordinates known, top-left and bottom-right, color, and width
        cv2.rectangle(grayscale, (x - offset, y - offset), (x+w+offset, y+h+offset), (255,255,255), 4)
        #Slice of frame to select out face part
        face_section = grayscale[y - offset : y + h + offset, x - offset : x + w + offset]
        #resizing face section to a particular standard so that we will be able to apply KNN later
        face_section = cv2.resize(face_section, (100,100))
        #Storing facial data
        face_data.append(face_section)
    #increment facecount, better to interpret as framecount or size of training data
    facecount += 1
    print(facecount)
    cv2.imshow("Face Detection", grayscale)
    #first and operation basically gets key from user and compares it to binary equivalent of mentioned key
    #loop breaks if key matches
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
#releases capture interface
capture.release()
#closes any windows created, not doing so will cause process to freeze
cv2.destroyAllWindows()

#convert tuple to numpy array
face_data = np.asarray(face_data)
#resize, flattening to 1d rows, -1 causes other dimensions to adjust in favor of mantaining specified ones
face_data = face_data.reshape((face_data.shape[0],-1))
print(np.shape(face_data))

#saving data to path
np.save(dataset_path + face_name, face_data)